In [170]:
import sympy as sp
from sympy import symbols, integrate, expand

# Basic

$x = J q$

In [171]:
xbars = symbols('\\bar{x}_1 \\bar{x}_2 \\bar{x}_3', real=True)
xbar = sp.Matrix(xbars)
xbar

Matrix([
[\bar{x}_1],
[\bar{x}_2],
[\bar{x}_3]])

In [172]:
J = sp.Matrix.zeros(3,12)
J[0:3,0:3] = sp.eye(3)
J[0,3:6] = xbar.T
J[1,6:9] = xbar.T
J[2,9:12] = xbar.T
J

Matrix([
[1, 0, 0, \bar{x}_1, \bar{x}_2, \bar{x}_3,         0,         0,         0,         0,         0,         0],
[0, 1, 0,         0,         0,         0, \bar{x}_1, \bar{x}_2, \bar{x}_3,         0,         0,         0],
[0, 0, 1,         0,         0,         0,         0,         0,         0, \bar{x}_1, \bar{x}_2, \bar{x}_3]])

In [173]:
x = symbols('x_1 x_2 x_3', real=True)
x = sp.Matrix(x)

J.T * x

Matrix([
[          x_1],
[          x_2],
[          x_3],
[\bar{x}_1*x_1],
[\bar{x}_2*x_1],
[\bar{x}_3*x_1],
[\bar{x}_1*x_2],
[\bar{x}_2*x_2],
[\bar{x}_3*x_2],
[\bar{x}_1*x_3],
[\bar{x}_2*x_3],
[\bar{x}_3*x_3]])

In [174]:
J.T * J

Matrix([
[        1,         0,         0,           \bar{x}_1,           \bar{x}_2,           \bar{x}_3,                   0,                   0,                   0,                   0,                   0,                   0],
[        0,         1,         0,                   0,                   0,                   0,           \bar{x}_1,           \bar{x}_2,           \bar{x}_3,                   0,                   0,                   0],
[        0,         0,         1,                   0,                   0,                   0,                   0,                   0,                   0,           \bar{x}_1,           \bar{x}_2,           \bar{x}_3],
[\bar{x}_1,         0,         0,        \bar{x}_1**2, \bar{x}_1*\bar{x}_2, \bar{x}_1*\bar{x}_3,                   0,                   0,                   0,                   0,                   0,                   0],
[\bar{x}_2,         0,         0, \bar{x}_1*\bar{x}_2,        \bar{x}_2**2, \bar{x}_2*\bar{x}_3

To integrate on the volume of a discrete Affine Body, we can:

1. using volume integral formula directly
2. using divergence theorem to convert the volume integral to a surface integral

We will do both way in this notebook.

First, we define the symbols we will use in the calculations:

In [175]:
p_0a, p_0b = symbols('p_0^alpha p_0^beta')
p_1a, p_1b = symbols('p_1^alpha p_1^beta')
p_2a, p_2b = symbols('p_2^alpha p_2^beta')
p_3a, p_3b = symbols('p_3^alpha p_3^beta')

e_1a, e_1b = symbols('e_1^alpha e_1^beta')
e_2a, e_2b = symbols('e_2^alpha e_2^beta')
e_3a, e_3b = symbols('e_3^alpha e_3^beta')

expr_e_1a, expr_e_1b = p_1a - p_0a, p_1b - p_0b
expr_e_2a, expr_e_2b = p_2a - p_0a, p_2b - p_0b
expr_e_3a, expr_e_3b = p_3a - p_0a, p_3b - p_0b


# Volume Integral

To integrate on the volume, we need user to provide a tetmesh of the body. With the tetmesh, we can directly calculate the volume of every tetrahedron and sum them up. Say we have $N$ tetrahedrons, the volume of the body is:

$$
V = \sum_i^N V_i = \sum_i^N \int_{\Omega_i} dV
$$

We do integration on each tetrahedron.

For a tetrahedron with vertices $p_0, p_1, p_2, p_3$, we define:

$$
e_1 = p_1 - p_0, e_2 = p_2 - p_0, e_3 = p_3 - p_0
$$

the parameterization of any point $p$ in the tetrahedron is:

$$
p = p_0 + e_1 u + e_2 v + e_3 w, u,v,w \in [0,1], u+v+w \le 1 
$$

The volume of the tetrahedron is:

$$
V_i = \int_{\Omega_i} dV = \int_{\Omega_i} det(e_1, e_2, e_3) du dv dw = det(e_1, e_2, e_3) \int_{\Omega_i} du dv dw
$$

So how to calculate the integral $\int_{\Omega_i} du dv dw$?

In [176]:
u,v,w = symbols('u v w')

fvw = integrate(1, (u, 0, 1-v-w))
fw = integrate(fvw, (v, 0, 1-w))
f = integrate(fw, (w, 0, 1))

f

1/6

$\frac{1}{6}$.

So the volume of the tetrahedron is:

$$
V_i = \frac{1}{6} det(e_1, e_2, e_3) = \frac{1}{6} e_1 \cdot (e_2 \times e_3)
$$

Is just how we calculate the tetrahedron volume geometrically!

So if we have a uniform mass density $\rho$, the mass of the tetrahedron is:

$$
m_i = \rho V_i = \frac{\rho}{6} e_1 \cdot (e_2 \times e_3)
$$

When we calculate the mass matrix of affine body, we not only need the `mass` but also other terms.

$$
(mx^\alpha) = \int_{\Omega} \rho x^\alpha dV = \sum_{i=1}^N \int_{\Omega_i} \rho x^\alpha dV, \alpha = 1,2,3
$$

and

$$
(mx^\alpha x^\beta) = \int_{\Omega} \rho x^\alpha x^\beta dV = \sum_{i=1}^N \int_{\Omega_i} \rho x^\alpha x^\beta dV, \alpha, \beta = 1,2,3
$$


Now we still using the uniform mass density. $x^\alpha$ is the $\alpha$-th component of the position vector.

So we can integrate on each tetrahedron, consider the parameterization of the tetrahedron, we have:

$$
\begin{align}
\rho \int_{\Omega_i} x^\alpha dV \\
= \rho \int_{\Omega_i} (p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) det(e_1,e_2,e_3) du dv dw \\
= \rho det(e_1,e_2,e_3) \int_{\Omega_i} (p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) du dv dw
\end{align}
$$

Something we don't know is the integral term, we can directly calculate it using sympy.

In [177]:
def integrate_V(f):
    fvw = integrate(f, (u, 0, 1-v-w))
    fw = integrate(fvw, (v, 0, 1-w))
    f = integrate(fw, (w, 0, 1))
    return f

integrate_V(p_0a + e_1a*u + e_2a*v + e_3a*w)

e_1^alpha/24 + e_2^alpha/24 + e_3^alpha/24 + p_0^alpha/6

In [178]:
integrate_V(p_0a + expr_e_1a*u + expr_e_2a*v + expr_e_3a*w)

p_0^alpha/24 + p_1^alpha/24 + p_2^alpha/24 + p_3^alpha/24

And let's go to the $(mx^\alpha x^\beta)$ term.

We imagine that we need to calculate the second order integral over $u,v,w$, so just like the first order integral, we can calculate the coefficients of the terms.

It time to expand the $\int_{\Omega_i} \rho x^\alpha x^\beta dV$ term.

$$
\begin{align}
\int_{\Omega_i} \rho x^\alpha x^\beta dV
= \rho det(e_1,e_2,e_3) \int_{\Omega_i}(p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) (p_0^\beta + e_1^\beta u + e_2^\beta v + e_3^\beta w) du dv dw
\end{align}
$$

In [179]:
P = (p_0a + e_1a * u + e_2a * v + e_3a * w) * (p_0b + e_1b * u + e_2b * v + e_3b * w)

integrate_V(P)

e_1^alpha*e_1^beta/60 + e_1^alpha*e_2^beta/120 + e_1^alpha*e_3^beta/120 + e_1^alpha*p_0^beta/24 + e_1^beta*e_2^alpha/120 + e_1^beta*e_3^alpha/120 + e_1^beta*p_0^alpha/24 + e_2^alpha*e_2^beta/60 + e_2^alpha*e_3^beta/120 + e_2^alpha*p_0^beta/24 + e_2^beta*e_3^alpha/120 + e_2^beta*p_0^alpha/24 + e_3^alpha*e_3^beta/60 + e_3^alpha*p_0^beta/24 + e_3^beta*p_0^alpha/24 + p_0^alpha*p_0^beta/6

In [180]:
P = (p_0a + expr_e_1a * u + expr_e_2a * v + expr_e_3a * w) * (p_0b + expr_e_1b * u + expr_e_2b * v + expr_e_3b * w)

integrate_V(P)

p_0^alpha*p_0^beta/60 + p_0^alpha*p_1^beta/120 + p_0^alpha*p_2^beta/120 + p_0^alpha*p_3^beta/120 + p_0^beta*p_1^alpha/120 + p_0^beta*p_2^alpha/120 + p_0^beta*p_3^alpha/120 + p_1^alpha*p_1^beta/60 + p_1^alpha*p_2^beta/120 + p_1^alpha*p_3^beta/120 + p_1^beta*p_2^alpha/120 + p_1^beta*p_3^alpha/120 + p_2^alpha*p_2^beta/60 + p_2^alpha*p_3^beta/120 + p_2^beta*p_3^alpha/120 + p_3^alpha*p_3^beta/60

For body force, say, we have a uniform force density $f \in R^3$, the body force we need to solve for the affine body related to:

$$
\int_{\Omega} f^\alpha dV = \sum_{i=1}^N \int_{\Omega_i} f^\alpha dV = \sum_{i=1}^N f^\alpha \int_{\Omega_i} dV
$$

and

$$
\begin{align}
  & \int_{\Omega} f^\alpha x^\alpha dV \\
= & \sum_{i=1}^N \int_{\Omega_i} f^\alpha x^\alpha dV \\
= & \sum_{i=1}^N f^\alpha \int_{\Omega_i} x^\alpha dV \\
= & \sum_{i=1}^N f^\alpha det(e_1,e_2,e_3) \int_{\Omega_i} (p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) du dv dw
\end{align}
$$

In [181]:
integrate_V(p_0a + e_1a*u + e_2a*v + e_3a*w)

e_1^alpha/24 + e_2^alpha/24 + e_3^alpha/24 + p_0^alpha/6

In [182]:
integrate_V(p_0a + expr_e_1a*u + expr_e_2a*v + expr_e_3a*w)

p_0^alpha/24 + p_1^alpha/24 + p_2^alpha/24 + p_3^alpha/24

# Surface Integral

We can also use the divergence theorem to convert the volume integral to a surface integral.

In this way, we need user to provide a closed triangle mesh of the body.

Say for triangle $i$, we have vertices $p_0, p_1, p_2$, we define:

$$
e_1 = p_1 - p_0, e_2 = p_2 - p_0
$$

and the corresponding parameterization of any point $p$ in the triangle is:

$$
p = p_0 + e_1 u + e_2 v, u,v \in [0,1], u+v \le 1
$$

The volume of the body is:

$$
V = \int_{\Omega} dV = \sum_{i=1}^N \int_{\Omega_i} dV = \sum_{i=1}^N \int_{\partial \Omega_i} \vec{F} \cdot d\vec{S},
$$

where $\nabla \cdot \vec{F} = 1$. We just choose $\vec{F} = p/3 = [x,y,z]^T/3$. 

Let's calculate the surface integral.

$$
\begin{align}
&\int_{\partial \Omega_i} \vec{F} \cdot d\vec{S} \\
&= \int_{\partial \Omega_i} \frac{1}{3} p \cdot d\vec{S} \\
&= \frac{1}{3} \int_{\partial \Omega_i} (p_0 + e_1 u + e_2 v) \cdot d\vec{S}\\
&= \frac{1}{3} p_0 \cdot (e_1 \times e_2) \int_{\partial \Omega_i} dudv
+ \cancel{\frac{1}{3} e_1 \cdot (e_1 \times e_2) \int_{\partial \Omega_i} u dudv}
+ \cancel{\frac{1}{3} e_2 \cdot (e_1 \times e_2) \int_{\partial \Omega_i} v dudv}\\
&= \frac{1}{3} p_0 \cdot {e_1 \times e_2} \int_{\partial \Omega_i} dudv \\
&= \frac{1}{3} det(p_0, e_1, e_2) \int_0^1 \int_0^{1-v}  dudv
\end{align}
$$

$$
(p_1 - p_0) \times (p_2 - p_0) = p_1 \times p_2 - p_1 \times p_0 - p_0 \times p_2 + p_0 \times p_0
= p_1 \times p_2 - p_1 \times p_0 - p_0 \times p_2
$$

$$
p_0 \cdot (p_1 \times p_2 - p_1 \times p_0 - p_0 \times p_2)
= p_0 \cdot (p_1 \times p_2) - p_0 \cdot (p_1 \times p_0) - p_0 \cdot (p_0 \times p_2)
= p_0 \cdot (p_1 \times p_2)
= det(p_0, p_1, p_2)
$$

Let's calculate the integral using sympy.

In [183]:
def integrate_S(f):
    fv = integrate(f, (u, 0, 1-v))
    f = integrate(fv, (v, 0, 1))
    return f

I_dudv = integrate_S(1)

I_dudv

1/2

So we have:

$$
\int_0^1 \int_0^{1-v}  dudv = \frac{1}{2}
$$

So the surface integral is:

$$
\int_{\partial \Omega_i} \vec{F} \cdot d\vec{S} = \frac{1}{6} det(p_0, e_1, e_2) = \frac{1}{6} det(p_0, p_1, p_2)
$$

Then we can calculate the mass matrix of the body (with uniform mass density) using the surface integral.

$$
m = \int_{\Omega} \rho dV = \sum_{i=1}^{N} \frac{1}{6} \rho det(p_0, p_1, p_2)_i
$$

And we can also calculate the $(mx^\alpha)$ and $(mx^\alpha x^\beta)$ terms.

$$
\begin{align}
& (mx^\alpha) \\
= & \int_{\Omega} \rho x^\alpha dV \\
= & \sum_{i=1}^N \int_{\Omega_i} \rho x^\alpha dV \\
= & \sum_{i=1}^N \int_{\partial \Omega_i} \vec{F}^\alpha d\vec{S} \\
\end{align}
$$

where $\nabla \cdot \vec{F}^\alpha = \rho p^\alpha$, we just choose:
 
$$
\vec{F}^\alpha = 
\frac{1}{2} \rho (p^\alpha)^2 \cdot \hat{i}^\alpha
$$

where $\hat{i}^\alpha$ is the $\alpha$-th unit vector:

$$
\hat{i}^1 = [1,0,0]^T, \hat{i}^2 = [0,1,0]^T, \hat{i}^3 = [0,0,1]^T
$$

Plug in the parameterization of the triangle, we have:

$$
\begin{align}
 &\int_{\partial \Omega_i} \vec{F}^\alpha \cdot d\vec{S} \\
=& \int_{\partial \Omega_i} \frac{1}{2} \rho (p^\alpha)^2 N^\alpha dudv \\
=& \frac{1}{2} \rho N^\alpha \int_{\partial \Omega_i} (p_0^\alpha  + e_1^\alpha  u + e_2^\alpha  v)^2 dudv
\end{align}
$$

In [184]:
p_a = p_0a + e_1a * u + e_2a * v

P = expand(p_a * p_a)

integrate_S(P)

e_1^alpha**2/12 + e_1^alpha*e_2^alpha/12 + e_1^alpha*p_0^alpha/3 + e_2^alpha**2/12 + e_2^alpha*p_0^alpha/3 + p_0^alpha**2/2

In [185]:
p_a = p_0a + expr_e_1a * u + expr_e_2a * v

P = expand(p_a * p_a)

integrate_S(P)

p_0^alpha**2/12 + p_0^alpha*p_1^alpha/12 + p_0^alpha*p_2^alpha/12 + p_1^alpha**2/12 + p_1^alpha*p_2^alpha/12 + p_2^alpha**2/12

$$
\begin{align}
& (mx^\alpha x^\beta) \\
= & \int_{\Omega} \rho x^\alpha x^\beta dV \\
= & \sum_{i=1}^N \int_{\Omega_i} \rho x^\alpha x^\beta dV \\
= &\sum_{i=1}^N \int_{\partial \Omega_i} \vec{F}^{\alpha\beta} d\vec{S}
\end{align}
$$

When $\alpha = \beta$, we choose:

$$
\vec{F}^{\alpha\alpha} = \frac{1}{3} \rho (p^\alpha)^3 \cdot \hat{i}^\alpha
$$

$$
\begin{align}
& \int_{\partial \Omega_i} \vec{F}^{\alpha\alpha} \cdot d\vec{S} \\
= & \int_{\partial \Omega_i} \frac{1}{3} \rho (p^\alpha)^3 N^\alpha dudv \\
= & \frac{1}{3} \rho N^\alpha \int_{\partial \Omega_i} (p_0^\alpha  + e_1^\alpha  u + e_2^\alpha  v)^3 dudv
\end{align}
$$

let's calculate the integrals using sympy.

In [186]:
P = expand(p_a**3)

integrate_S(P)

p_0^alpha**3/20 + p_0^alpha**2*p_1^alpha/20 + p_0^alpha**2*p_2^alpha/20 + p_0^alpha*p_1^alpha**2/20 + p_0^alpha*p_1^alpha*p_2^alpha/20 + p_0^alpha*p_2^alpha**2/20 + p_1^alpha**3/20 + p_1^alpha**2*p_2^alpha/20 + p_1^alpha*p_2^alpha**2/20 + p_2^alpha**3/20

If $\alpha \neq \beta$, we choose:

$$
\vec{F}^{\alpha\beta} = \frac{1}{2} \rho (p^\alpha)^2 p^\beta \cdot \hat{i}^\alpha
$$

E.g. for $\alpha = x, \beta = y$

$$
\vec{F}^{x,y} = 
\begin{bmatrix}
\frac{1}{2} \rho x^2 y \\ 
0 \\
0 \\
\end{bmatrix}
$$

So we have:

$$
\begin{align}
& \int_{\partial \Omega_i} \vec{F}^{\alpha\beta} \cdot d\vec{S} \\
= & \int_{\partial \Omega_i} \frac{1}{2} \rho (p^\alpha)^2 p^\beta N^\alpha dudv \\
= & \frac{1}{2} \rho N^\alpha \int_{\partial \Omega_i} (p_0^\alpha  + e_1^\alpha  u + e_2^\alpha  v)^2 (p_0^\beta  + e_1^\beta  u + e_2^\beta  v) dudv
\end{align}
$$

we can calculate the integrals using sympy.

In [187]:
p_a = p_0a + e_1a * u + e_2a * v
p_b = p_0b + e_1b * u + e_2b * v


P = expand(p_a**2 * p_b)

integrate_S(P)

e_1^alpha**2*e_1^beta/20 + e_1^alpha**2*e_2^beta/60 + e_1^alpha**2*p_0^beta/12 + e_1^alpha*e_1^beta*e_2^alpha/30 + e_1^alpha*e_1^beta*p_0^alpha/6 + e_1^alpha*e_2^alpha*e_2^beta/30 + e_1^alpha*e_2^alpha*p_0^beta/12 + e_1^alpha*e_2^beta*p_0^alpha/12 + e_1^alpha*p_0^alpha*p_0^beta/3 + e_1^beta*e_2^alpha**2/60 + e_1^beta*e_2^alpha*p_0^alpha/12 + e_1^beta*p_0^alpha**2/6 + e_2^alpha**2*e_2^beta/20 + e_2^alpha**2*p_0^beta/12 + e_2^alpha*e_2^beta*p_0^alpha/6 + e_2^alpha*p_0^alpha*p_0^beta/3 + e_2^beta*p_0^alpha**2/6 + p_0^alpha**2*p_0^beta/2

In [188]:
p_a = p_0a + expr_e_1a * u + expr_e_2a * v
p_b = p_0b + expr_e_1b * u + expr_e_2b * v

P = expand(p_a**2 * p_b)

integrate_S(P)

p_0^alpha**2*p_0^beta/20 + p_0^alpha**2*p_1^beta/60 + p_0^alpha**2*p_2^beta/60 + p_0^alpha*p_0^beta*p_1^alpha/30 + p_0^alpha*p_0^beta*p_2^alpha/30 + p_0^alpha*p_1^alpha*p_1^beta/30 + p_0^alpha*p_1^alpha*p_2^beta/60 + p_0^alpha*p_1^beta*p_2^alpha/60 + p_0^alpha*p_2^alpha*p_2^beta/30 + p_0^beta*p_1^alpha**2/60 + p_0^beta*p_1^alpha*p_2^alpha/60 + p_0^beta*p_2^alpha**2/60 + p_1^alpha**2*p_1^beta/20 + p_1^alpha**2*p_2^beta/60 + p_1^alpha*p_1^beta*p_2^alpha/30 + p_1^alpha*p_2^alpha*p_2^beta/30 + p_1^beta*p_2^alpha**2/60 + p_2^alpha**2*p_2^beta/20

When we calculate the body force, we need to solve for the affine body related to $\int_{\Omega} f^\alpha dV$ and $\int_{\Omega} f^\alpha x^\alpha dV$. 

$$
\begin{align}
  & \int_{\Omega} f^\alpha dV \\
= & \sum_{i=1}^N \int_{\Omega_i} f^\alpha dV \\ 
= & \sum_{i=1}^N f^\alpha \int_{\Omega_i} dV \\
= & \sum_{i=1}^N f^\alpha \int_{\partial \Omega_i} \vec{F} d\vec{S}
\end{align}
$$

we choose $\vec{F} = \frac{1}{3} p \cdot \hat{i}^\alpha$.

$$
\sum_{i=1}^N f^\alpha \int_{\partial \Omega_i} \vec{F} d\vec{S} = \sum_{i=1}^N \frac{1}{6} f^\alpha det(p_0, e_1, e_2)_i
$$


$$
\begin{align}
  & \int_{\Omega} f^\alpha x^\alpha dV \\
= & \sum_{i=1}^N \int_{\Omega_i} f^\alpha x^\alpha dV \\
= & \sum_{i=1}^N f^\alpha \int_{\Omega_i} x^\alpha dV \\
= & \sum_{i=1}^N f^\alpha \int_{\partial \Omega_i} \vec{F}^\alpha d\vec{S}
= & \sum_{i=1}^N \frac{1}{2}  f^\alpha N^\alpha \int_{\partial \Omega_i} (p_0^\alpha  + e_1^\alpha  u + e_2^\alpha  v)^2 dudv
\end{align}
$$

We choose $\vec{F}^\alpha = \frac{1}{2} (p^\alpha)^2 \cdot \hat{i}^\alpha$.

In [189]:
p_a = p_0a + e_1a * u + e_2a * v

P = expand(p_a * p_a)

integrate_S(P)

e_1^alpha**2/12 + e_1^alpha*e_2^alpha/12 + e_1^alpha*p_0^alpha/3 + e_2^alpha**2/12 + e_2^alpha*p_0^alpha/3 + p_0^alpha**2/2

In [190]:
p_a = p_0a + expr_e_1a * u + expr_e_2a * v
P = expand(p_a * p_a)

integrate_S(P)

p_0^alpha**2/12 + p_0^alpha*p_1^alpha/12 + p_0^alpha*p_2^alpha/12 + p_1^alpha**2/12 + p_1^alpha*p_2^alpha/12 + p_2^alpha**2/12